# Nest

Prepares data for the Nest page 

In [55]:
import os
import pandas as pd
from events import (
    load_event_data, filter_public_events, filter_by_project, prepare, save_files, count_wards, filter_signature_events
)
import ticket_helpers


Set output directory

In [56]:
OUT_DIR = '../../../../docs/metrics/events/nest/_data/events'
os.makedirs(OUT_DIR, exist_ok=True)

Read in event data and apply Airtable filters.

* Project_type is 'Produce (Signature)'
* Project_name is '10 - Melt'
* Event_type is 'Public Event -' and 'CLE - Community Engagement'

In [57]:
data = (
  load_event_data()
    .pipe(filter_signature_events)
    .pipe(filter_by_project, '10 - Melt')
    .pipe(filter_public_events)
  )
data

,actual_audience_size_number_of_participants_in_person,actual_audience_size_number_of_participants_online,airtable_id,end_date,event_name,event_type,event_unique_identifier,number_of_booked_participants,postcode_(from_venue),project_name,project_type,season,start_date,ticket_tailor_id,ward_(from_venue)
1074,1338.0,NaN,241,2023-09-09,Melt (Run),['Public Event - Performance'],10 - Melt: Melt (Run),1542.0,['LS26 8AL'],10 - Melt,Produce (Signature),['Season 3 - Dreaming'],2023-09-05,"ev_2809632, ev_2809633, ev_2809634, ev_28096...",['Garforth and Swillington']


Run the processing and save the files

In [58]:
data = data.pipe(filter_public_events).pipe(prepare).pipe(save_files, OUT_DIR)

## Ticket processing

Extract the ticket data

In [59]:
ticket_sales = data.pipe(filter_public_events).pipe(ticket_helpers.get_tickets_for_event)
ticket_sales

,ticket_tailor_id,airtable_id,created_at,event_date,number_of_tickets,ward_code
0,ev_2809632,241,2023-07-11 21:55:42,2023-09-05,2,E05011402
1,ev_2809632,241,2023-07-13 13:29:16,2023-09-05,2,E05011402
2,ev_2809632,241,2023-07-14 18:21:57,2023-09-05,3,E05001372
3,ev_2809632,241,2023-07-17 11:10:07,2023-09-05,2,E05011403
4,ev_2809632,241,2023-07-17 16:39:03,2023-09-05,5,E05011390
...,...,...,...,...,...,...
715,ev_2809636,241,2023-09-08 11:43:24,2023-09-09,4,E05012841
716,ev_2809636,241,2023-09-08 15:24:13,2023-09-09,2,E05011413
717,ev_2809636,241,2023-09-08 15:58:49,2023-09-09,1,NaN
718,ev_2809636,241,2023-09-09 11:43:24,2023-09-09,2,E05011396


In [60]:
pd.DataFrame({
  'tickets': ticket_sales.groupby('event_date').number_of_tickets.sum().astype(int),
}).resample('D').sum().reset_index().to_csv(os.path.join(OUT_DIR, 'tickets_by_event_date.csv'), index=False)

By Date

In [61]:
ticket_sales.pipe(ticket_helpers.summarise_by_ward).reset_index().to_csv(
    os.path.join(OUT_DIR, 'tickets_by_ward.csv'), index=False)


By Date

In [62]:
(
  ticket_sales
    .pipe(ticket_helpers.summarise_by_date)
    .resample('W-FRI').sum()
    .cumsum()
)
ticket_sales.to_csv(os.path.join(OUT_DIR, 'tickets_by_date.csv'))